In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass#print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Download Youtube videos


In [2]:
! pip install pytube
import pandas as pd
import numpy as np
import cv2
import os
from pytube import YouTube
import moviepy.editor as mp
from pydub import AudioSegment
import speech_recognition as sr

#paths for training and lables file in dataset

train_text = r'/kaggle/input/sports-1m-dataset/original/train_partition.txt'
labels_text=r'/kaggle/input/sports-1m-dataset/labels.txt'

#creating dataframe with the text data

df = pd.read_csv(train_text, sep='\s+', header=None, names=['URL', 'ID'])
labels = pd.read_csv(labels_text,header=None,names=['Games'])
#print(df[:5])
c= df['URL'][7]
print(c)
#print(labels.iloc[int(c)])
frames =[df,labels]
#print(labels[:3])
combined =pd.concat(frames,ignore_index=True)

#downloading the video from the link 

def download_video(video_link, output_folder='videos'):
    try:
        yt = YouTube(video_link)
        chosen_stream = yt.streams.filter(res="360p", file_extension="mp4").first()
        chosen_stream.download(output_path=output_folder)
        print(f'Video downloaded successfully')
    except Exception as e:
        print(f'Error downloading video: {e}')
        
output_folder = 'videos'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(1):
    link = df['URL'][i]
    download_video(link)
    
def delete_videos(video_folder='videos'):
    try:
        # List all files in the video folder
        video_files = os.listdir(video_folder)

        # Iterate through the files and delete each one
        for video_file in video_files:
            video_path = os.path.join(video_folder, video_file)
            os.remove(video_path)

        # Remove the video folder itself
        os.rmdir(video_folder)

        print(f'Videos deleted successfully')
    except Exception as e:
        print(f'Error deleting videos: {e}')
        
def extract_captions(video_path, output_text_file='captions.txt'):
    try:
        # Convert video to audio
        audio = mp.VideoFileClip(video_path).audio
        audio.export("temp_audio.wav", format="wav")

        # Load audio file
        recognizer = sr.Recognizer()
        with sr.AudioFile("temp_audio.wav") as source:
            audio_data = recognizer.record(source)

        # Perform speech recognition
        captions = recognizer.recognize_google(audio_data)

        # Save captions to text file
        with open(output_text_file, 'w') as text_file:
            text_file.write(captions)

        print(f'Captions extracted successfully and saved to {output_text_file}')
    except Exception as e:
        print(f'Error extracting captions: {e}')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
https://www.youtube.com/watch?v=AZM37_FqL5w
Video downloaded successfully


In [3]:
# Extracting frames from the Video and then saving the pictures
def extract_frames(video_path, output_folder='frames', frame_interval=5):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frames per second (fps) and total number of frames
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval based on the specified interval
    frame_interval = max(1, int(fps * frame_interval))

    # Loop through the frames and save every nth frame
    for frame_num in range(0, total_frames, frame_interval):
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)

        # Read the frame
        ret, frame = cap.read()

        # Save the frame as an image
        if ret:
            frame_path = os.path.join(output_folder, f'frame_{frame_num}.jpg')
            cv2.imwrite(frame_path, frame)

    # Release the video capture object
    cap.release()

    print(f'Frames extracted successfully. Check the \'{output_folder}\' folder.')

 # Training the Model

In [4]:
import tensorflow as tf
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Input, Layer
from tensorflow.keras.layers import Embedding, LSTM, add, Concatenate, Reshape, concatenate, Bidirectional
from tensorflow.keras.applications import VGG16, ResNet50, DenseNet201
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
warnings.filterwarnings('ignore')


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Path of the images and text file
images_path = '/kaggle/working/frames'
data =pd.read_csv('/kaggle/working/frames/captions.txt')
#data.head()

In [6]:
def read_image(path,img_size=224):
    img =load_img(path,color_mode='rgb',target_size=(img_size,img_size))
    img = img_to_array(img)
    img =img/255
    return img

In [7]:
def display_imgs(temp_df):
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize = (20,20))
    n=0
    for i in range(15):
        n+=1
        plt.subplot(5,5,n)
        plt.subplots_adjust(hspace=0.7,wspace=0.3)
        image = read_image(f'{images_path}/{temp_df.image[i]}')
        plt.imshow(image)
        plt.title('\n'.join(wrap(temp_df.caption[i],20)))
        plt.axis('off')

In [8]:
def text_processing(data):
    data['caption'] = data['caption'].apply(lambda x:x.lower())
    data['caption'] = data['caption'].apply(lambda x:x.replace("[^A-Za-z]",""))
    data['caption'] = data['caption'].apply(lambda x:x.replace("\s+"," "))
    data['caption'] = data['caption'].apply(lambda x:" ".join([word for word in x.split() if len(word)>1]))
    data['caption'] = 'startseq '+data['caption']+' endseq'
    
    return data
data_processed = text_processing(data)


In [9]:
captions =data_processed['caption'].tolist()
captions[0]

'startseq child in pink dress is climbing up set of stairs in an entry way endseq'

In [10]:

# Creating a word tokenizer
tokenizer =Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index)+1

# creating max_length param to gert the padding length
max_length = max(len(caption.split()) for caption in captions)

images = data['image'].unique().tolist()
#print(images)
n_images =len(images)
split_index = round(0.85*n_images)

# splitting data for training and testing
train_images = images[:split_index]
test_images = images[split_index:]

train = data[data['image'].isin(train_images)]
test =data[data['image'].isin(test_images)]

train.reset_index(inplace =True,drop =True)
test.reset_index(inplace =True,drop =True)
#tokenizer.texts_to_sequences([captions[8]])[0]

In [11]:
model =DenseNet201()
fe = Model(inputs =model.input,outputs =model.layers[-2].output)

82524592/82524592 [==============================] - 0s 0us/step


In [12]:
# feature extraction from the images using DenseNet201.

def ext_features():
    features = {}
    for image in tqdm(images):
        img = read_image(f'/kaggle/working/frames{image}')
        img =np.expand_dims(img,axis=0)
        feature =fe.predict(img,verbose=0)
        features[image] =feature
    return features

In [13]:
pickle_file = 'features.pickle'
    
# Check if the pickle file already exists
if os.path.exists(pickle_file):
    with open(pickle_file, 'rb') as f:
        features = pickle.load(f)
else:
    features = ext_features()
    with open(pickle_file, 'wb') as f:
            pickle.dump(features, f)


100%|██████████| 8091/8091 [13:24<00:00, 10.06it/s]


In [14]:
#print(features['1000268201_693b08cb0e.jpg'])

# Extracting individual features

def ind_features(path,image):
    features ={}
    img =read_image(f'{path}/{image}')
    img =np.expand_dims(img,axis=0)
    feature =fe.predict(img,verbose=0)
    features[image] =feature
    return features

In [15]:
class CustomDataGeneratot(Sequence):
    def __init__(self,df,X_col,y_col,batch_size,directory,tokenizer,vocab_size,max_length,features,shuffle=True):
        self.df=df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.directory = directory
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.features = features
        self.shuffle = shuffle
        self.n = len(self.df)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df =self.df.sample(frac =1).reset_index(drop=True)
    def __len__(self):
        return self.n//self.batch_size
    def __getitem__(self,index):
        batch =self.df.iloc[index*self.batch_size:(index+1)*self.batch_size,:]
        X1,X2,y =self.__get_data(batch)
        return (X1,X2),y
    def __get_data(self,batch):
        
        X1, X2, y = list(), list(), list()
        
        images = batch[self.X_col].tolist()
           
        for image in images:
            feature = self.features[image][0]
            
            captions = batch.loc[batch[self.X_col]==image, self.y_col].tolist()
            for caption in captions:
                seq = self.tokenizer.texts_to_sequences([caption])[0]

                for i in range(1,len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes=self.vocab_size)[0]
                    X1.append(feature)
                    X2.append(in_seq)
                    y.append(out_seq)
            
        X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                
        return X1, X2, y


In [16]:
input1 = Input(shape=1920,)
input2 = Input(shape=(max_length,))

img_features = Dense(256,activation='relu')(input1)
img_features_reshaped =Reshape((1,256),input_shape=(256,))(img_features)

sentence_features = Embedding(vocab_size,256,mask_zero=False)(input2)
merged = concatenate([img_features_reshaped,sentence_features],axis=1)
sentence_features =LSTM(256)(merged)

x=Dropout(0.5)(sentence_features)
x=add([x,img_features])
x=Dense(128,activation ='relu')(x)
x=Dropout(0.5)(x)
output =Dense(vocab_size,activation='softmax')(x)
caption_model =Model(inputs=[input1,input2],outputs =output)
caption_model.compile(loss='categorical_crossentropy',optimizer='adam')

In [17]:
train_generator =CustomDataGeneratot(df=train,X_col='image',y_col='caption',batch_size =64,directory=images_path,tokenizer=tokenizer,vocab_size=vocab_size,max_length=max_length,features=features)
validation_generator = CustomDataGeneratot(df=test,X_col='image',y_col='caption',batch_size =64,directory=images_path,tokenizer=tokenizer,vocab_size=vocab_size,max_length=max_length,features=features)


In [18]:
model_name = 'image_captioning_model.h5'

checkpoint = ModelCheckpoint(model_name,
                            monitor='val_loss',
                            mode='min',
                            save_best_only =True,
                            verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

In [19]:
training = caption_model.fit(train_generator,
                            epochs=100,
                            validation_data=validation_generator,
                            callbacks=[checkpoint,learning_rate_reduction])

Epoch 1/100
537/537 [==============================] - ETA: 0s - loss: 5.1287
Epoch 1: val_loss improved from inf to 4.23467, saving model to image_captioning_model.h5
537/537 [==============================] - 220s 402ms/step - loss: 5.1287 - val_loss: 4.2347 - lr: 0.0010
Epoch 2/100
537/537 [==============================] - ETA: 0s - loss: 4.1822
Epoch 2: val_loss improved from 4.23467 to 3.91157, saving model to image_captioning_model.h5
537/537 [==============================] - 54s 100ms/step - loss: 4.1822 - val_loss: 3.9116 - lr: 0.0010
Epoch 3/100
537/537 [==============================] - ETA: 0s - loss: 3.9178
Epoch 3: val_loss improved from 3.91157 to 3.77698, saving model to image_captioning_model.h5
537/537 [==============================] - 49s 92ms/step - loss: 3.9178 - val_loss: 3.7770 - lr: 0.0010
Epoch 4/100
537/537 [==============================] - ETA: 0s - loss: 3.7580
Epoch 4: val_loss improved from 3.77698 to 3.70204, saving model to image_captioning_model.h5
5

In [20]:
model_path = '/kaggle/input/modelll/image_captioning_model (2).h5'
loaded_model = tf.keras.models.load_model(model_path)

In [21]:
def retokenizer(integer,tokenizer):
    for word, index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None

In [22]:
def remove_start_end_tokens(sentence):
    start_token = 'startseq'
    end_token = 'endseq'

    # Check if both start and end tokens are present in the sentence
    if start_token in sentence and end_token in sentence:
        # Remove both start and end tokens
        cleaned_sentence = sentence.replace(start_token, '').replace(end_token, '')
        # Remove extra whitespaces
        cleaned_sentence = ' '.join(cleaned_sentence.split())
        return cleaned_sentence
    else:
        # If start or end token is not found, return the original sentence
        return sentence

In [23]:
def predict_caption(model, image, tokenizer, max_length, features):
    
    feature = features[image]
    in_text = "startseq"
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], max_length)

        y_pred = model.predict([feature,sequence])
        y_pred = np.argmax(y_pred)
        
        word = retokenizer(y_pred, tokenizer)
        
        if word is None:
            break
            
        in_text+= " " + word
        
        if word == 'endseq':
            break

    in_text =remove_start_end_tokens(in_text)
       
    return in_text 

In [24]:
samples = test.sample(15)
#print(samples[:3])
samples.reset_index(drop=True,inplace=True)


In [25]:
for index , record in samples.iterrows():
    img = load_img(os.path.join(images_path,record['image']),target_size =(224,224))
    img =img_to_array(img)
    img =img/255
    caption = predict_caption(loaded_model,record['image'],tokenizer,max_length,features)
    
    samples.loc[index,'caption'] = caption

1/1 [==============================] - 0s 18ms/step


In [26]:
video = r'/kaggle/working/videos/Capoeira regional Cuba Habana2.mp4'
extract_frames(video)

Frames extracted successfully. Check the 'frames' folder.


[h264 @ 0x5cd8788624c0] Missing reference picture, default is 65562
[h264 @ 0x5cd87e610980] mmco: unref short failure
[h264 @ 0x5cd87e610980] mmco: unref short failure


In [27]:
path = '/kaggle/working/frames'
vid_images = os.listdir(path)
print(vid_images)

['frame_1250.jpg', 'frame_625.jpg', 'frame_750.jpg', 'frame_1000.jpg', 'frame_1375.jpg', 'frame_500.jpg', 'frame_125.jpg', 'frame_1625.jpg', 'frame_1125.jpg', 'frame_1750.jpg', 'frame_375.jpg', 'frame_250.jpg', 'frame_0.jpg', 'frame_1500.jpg', 'frame_875.jpg']


In [28]:
def ext_features(path):
    features = {}
    for image in tqdm(vid_images):
        img = read_image(f'{path}/{image}')
        img =np.expand_dims(img,axis=0)
        feature =fe.predict(img,verbose=0)
        features[image] =feature
    return features

In [29]:
vid_features = ext_features(path)

100%|██████████| 15/15 [00:01<00:00, 11.26it/s]


In [30]:
gen_captions = {}
for image in vid_images:
    vid_captions = predict_caption(loaded_model,image,tokenizer,max_length,vid_features)
    gen_captions[image] = vid_captions

1/1 [==============================] - 0s 19ms/step


In [31]:
# Assuming gen_captions is the dictionary containing image names and generated captions
gen_captions_df = pd.DataFrame(list(gen_captions.items()), columns=['image', 'caption'])
#print(gen_captions_df['caption'])

In [32]:
def display_imgs(temp_df,path):
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize = (20,20))
    n=0
    for i in range(15):
        n+=1
        plt.subplot(5,5,n)
        plt.subplots_adjust(hspace=0.7,wspace=0.3)
        image = read_image(f'{path}/{temp_df.image[i]}')
        plt.imshow(image)
        plt.title('\n'.join(wrap(temp_df.caption[i],20)))
        plt.axis('off')

# creating the words from the captions

In [36]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


all_captions = ' '.join(gen_captions_df['caption'])
tokens = word_tokenize(all_captions)
stop_words = set(stopwords.words("english"))
filtered_tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
print(filtered_tokens)
print(set(filtered_tokens))

['man', 'white', 'shirt', 'standing', 'front', 'street', 'group', 'people', 'standing', 'front', 'street', 'group', 'people', 'standing', 'front', 'building', 'man', 'blue', 'shirt', 'standing', 'front', 'street', 'man', 'black', 'shirt', 'white', 'shirt', 'standing', 'front', 'building', 'group', 'people', 'standing', 'front', 'building', 'two', 'men', 'white', 'white', 'shirt', 'standing', 'front', 'water', 'group', 'people', 'standing', 'front', 'building', 'group', 'people', 'standing', 'front', 'building', 'man', 'white', 'shirt', 'standing', 'front', 'building', 'group', 'people', 'standing', 'front', 'crowd', 'man', 'white', 'shirt', 'standing', 'front', 'man', 'two', 'men', 'blue', 'shirt', 'white', 'shirt', 'standing', 'front', 'air', 'group', 'people', 'standing', 'front', 'building', 'man', 'white', 'shirt', 'standing', 'front', 'crowd', 'people']
{'two', 'men', 'air', 'black', 'crowd', 'standing', 'front', 'building', 'group', 'blue', 'water', 'white', 'street', 'people', '

# CrossValidation


In [34]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Your list of input words
input_words = filtered_tokens

# Create a prompt with the input words
prompt = f"Generate words similar to: {', '.join(input_words)}."

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate text using the GPT-2 model
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Extract words from the generated text
generated_words = generated_text.split()

# Print the generated words
print(generated_words)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Generate', 'words', 'similar', 'to:', 'man,', 'white,', 'shirt,', 'standing,', 'front,', 'street,', 'group,', 'people,', 'standing,', 'front,', 'street,', 'group,', 'people,', 'standing,', 'front,', 'building,', 'man,', 'blue,', 'shirt,', 'standing,', 'front,', 'street,', 'man,', 'black,', 'shirt,', 'white,', 'shirt,', 'standing,', 'front,', 'building,', 'group,', 'people,', 'standing,', 'front,', 'building,', 'two,', 'men,', 'white,', 'white,', 'shirt,', 'standing,', 'front,', 'water,', 'group,', 'people,', 'standing,', 'front,', 'building,', 'group,', 'people,', 'standing,', 'front,', 'building,', 'man,', 'white,', 'shirt,', 'standing,', 'front,', 'building,', 'group,', 'people,', 'standing,', 'front,', 'crowd,', 'man,', 'white,', 'shirt,', 'standing,', 'front,', 'man,', 'two,', 'men,', 'blue,', 'shirt,', 'white,', 'shirt,', 'standing,', 'front,', 'air,', 'group,', 'people,', 'standing,', 'front,', 'building,', 'man,', 'white,', 'shirt,', 'standing,', 'front,', 'crowd,', 'people.']

In [ ]:
​
module readdata
      contains
      subroutine readdcd(maxframes,maxatoms,x,y,z,xbox,ybox,zbox,natoms,nframes)
      integer,device i,j
      integer,device maxframes,maxatoms
​
      double precision d(6),xbox,ybox,zbox
      real*4, allocatable,device   :: x(:,:)
      real*4, allocatable,device   :: y(:,:)
      real*4, allocatable,device  :: z(:,:)
​
      real*4 dummyr
      integer*4 nset, natoms, dummyi,nframes,tframes
      character*4 dummyc
      
      open(10,file='alk.traj.dcd',status='old',form='unformatted')
      read(10) dummyc, tframes,(dummyi,i=1,8),dummyr, (dummyi,i=1,9)
      read(10) dummyi, dummyr,dummyr
      read(10) natoms
      print*,"Total number of frames and atoms are",tframes,natoms
​
      allocate ( x(natoms,maxframes) )
      allocate ( y(natoms,maxframes) )
      allocate ( z(natoms,maxframes) )
​
      do j = 1,nframes
           read(10) (d(i),i=1, 6)
              
           read(10) (x(i,j),i=1,natoms)
           read(10) (y(i,j),i=1,natoms)
           read(10) (z(i,j),i=1,natoms)
      end do
      
      xbox=d(1)
      ybox=d(3)
      zbox=d(6)
      
      print*,"File reading is done: xbox,ybox,zbox",xbox,ybox,zbox
      return
​
      end subroutine readdcd
 end module readdata
​
program rdf
      use readdata
      use nvtx
      implicit none
      integer n,i,j,iconf,ind
      integer natoms,nframes,nbin
      integer maxframes,maxatoms
      parameter (maxframes=10,maxatoms=60000,nbin=2000)
      real*4, allocatable   :: x(:,:)
      real*4, allocatable   :: y(:,:)
      real*4, allocatable   :: z(:,:)
      double precision dx,dy,dz
      double precision xbox,ybox,zbox,cut
      double precision vol,r,del,s2,s2bond
      double precision, allocatable   ::  g(:)
      double precision rho,gr,lngr,lngrbond,pi,const,nideal,rf
      double precision rlower,rupper
      character  atmnm*4
      real*4 start,tread,tpairs,finish
        
      open(23,file='RDF.dat',status='unknown')
      open(24,file='Pair_entropy.dat',status='unknown')
​
      nframes=10
         
      call cpu_time(start)
​
      print*,"Going to read coordinates"
      call nvtxStartRange("Read File")
      call readdcd(maxframes,maxatoms,x,y,z,xbox,ybox,zbox,natoms,nframes)
      call nvtxEndRange
​
      call cpu_time(tread)
      allocate ( g(nbin) )
      g = 0.0d0
 
      pi=dacos(-1.0d0)
      vol=xbox*ybox*zbox
      rho=dble(natoms)/vol
​
      del=xbox/dble(2.0*nbin)
      write(*,*) "bin width is : ",del
      cut = dble(xbox * 0.5);
​
      !pair calculation
      call nvtxStartRange("Pair Calculation")
      do iconf=1,nframes
         if (mod(iconf,1).eq.0) print*,iconf
         do i=1,natoms
            do j=1,natoms
               dx=x(i,iconf)-x(j,iconf)
               dy=y(i,iconf)-y(j,iconf)
               dz=z(i,iconf)-z(j,iconf)
​
               dx=dx-nint(dx/xbox)*xbox
               dy=dy-nint(dy/ybox)*ybox
               dz=dz-nint(dz/zbox)*zbox
   
               r=dsqrt(dx**2+dy**2+dz**2)
               if(r<cut)then
                  ind=int(r/del)+1
                  g(ind)=g(ind)+1.0d0
               endif
            enddo
         enddo
      enddo
      call nvtxEndRange
​
      call cpu_time(tpairs)
      !entropy calculation
      s2=0.01d0
      s2bond=0.01d0 
      const=(4.0d0/3.0d0)*pi*rho
      call nvtxStartRange("Entropy Calculation")
      do i=1,nbin
          rlower=dble((i-1)*del)
          rupper=rlower+del
          nideal=const*(rupper**3-rlower**3)
          g(i)=g(i)/(dble(nframes)*dble(natoms)*nideal)
          r=dble(i)*del
          if (r.lt.2.0) then
            gr=0.0
          else
            gr=g(i)
          endif
​
          if (gr.lt.1e-5) then
            lngr=0.0
          else
            lngr=dlog(gr)
          endif
          if (g(i).lt.1e-6) then
            lngrbond=0.01
          else
            lngrbond=dlog(g(i))
          endif
​
          s2=s2-2*pi*rho*((gr*lngr)-gr+1)*del*r**2.0
          s2bond=s2bond-2*pi*rho*((g(i)*lngrbond)-g(i)+1)*del*r*r
​
          
          rf=dble(i-.5)*del
          write(23,*) rf,g(i)
      enddo
      call nvtxEndRange
​
      write(24,*)"s2      : ",s2
      write(24,*)"s2bond  : ",s2bond
      call cpu_time(finish)
      print*,"Read time:    ", tread-start
      print*,"Pairs time:   ", tpairs-tread
      print*,"Entropy time: ", finish-tpairs
      print*,"Total time:   ", finish-start
​
      deallocate(x,y,z,g)
end

In [ ]:
! Copyright (c) 2021 NVIDIA Corporation. All rights reserved.
module readdata
    contains
    subroutine readdcd(maxframes, maxatoms, x, y, z, xbox, ybox, zbox, natoms, nframes)
        integer i, j
        integer, intent(in) :: maxframes, maxatoms
        double precision, intent(out) :: xbox, ybox, zbox
        real(4), allocatable, intent(out) :: x(:,:)
        real(4), allocatable, intent(out) :: y(:,:)
        real(4), allocatable, intent(out) :: z(:,:)

        real(4), dimension(6) :: dummyr  ! Fix: dummyr is now an array

        integer :: nset, dummyi, tframes
        character(4) :: dummyc

        open(10, file='alk.traj.dcd', status='old', form='unformatted')
        read(10) dummyc, tframes, (dummyi, i=1, 8), dummyr, (dummyi, i=1, 9)
        read(10) dummyi, dummyr, dummyr
        read(10) natoms
        print*, "Total number of frames and atoms are", tframes, natoms
        allocate(x(natoms, maxframes))
        allocate(y(natoms, maxframes))
        allocate(z(natoms, maxframes))
        do j = 1, tframes  ! Corrected variable name from nframes to tframes
            read(10) (x(i, j), i=1, natoms)
            read(10) (y(i, j), i=1, natoms)
            read(10) (z(i, j), i=1, natoms)
        end do

        xbox = dummyr(1)
        ybox = dummyr(3)
        zbox = dummyr(6)

        print*, "File reading is done: xbox, ybox, zbox", xbox, ybox, zbox
        close(10)
    end subroutine readdcd
end module readdata

module rdfcuda
    use cudafor
    contains
    attributes(global) subroutine pair_calculation(x, y, z, nframes, natoms, nbin, g, xbox, del)
        ! Adjust the argument list to match the call
        integer, intent(in) :: nframes, natoms, nbin
        real(4), intent(in) :: x(natoms, nframes), y(natoms, nframes), z(natoms, nframes)
        real(8), intent(out) :: g(nbin)
        real(8), intent(in) :: xbox, del
        integer :: iconf, i, j, ind
        double precision :: dx, dy, dz, r, cut

        cut = xbox * 0.5d0
        do iconf = 1, nframes
            do i = 1, natoms
                do j = 1, natoms
                    dx = x(i, iconf) - x(j, iconf)
                    dy = y(i, iconf) - y(j, iconf)
                    dz = z(i, iconf) - z(j, iconf)
                    dx = dx - nint(dx / xbox) * xbox
                    dy = dy - nint(dy / xbox) * xbox
                    dz = dz - nint(dz / xbox) * xbox

                    r = sqrt(dx**2 + dy**2 + dz**2)
                    if (r < cut) then
                        ind = int(r / del) + 1
                        atomicAdd(g(ind), 1.0d0)
                    endif
                end do
            end do
        end do
    end subroutine pair_calculation

    attributes(global) subroutine entropy_calculation(g, nbin, rho, del, s2, s2bond)
        ! ... (unchanged)
    end subroutine entropy_calculation
end module rdfcuda

program rdf
    use readdata
    use rdfcuda

    implicit none
    integer, parameter :: maxframes = 10, maxatoms = 60000, nbin = 2000
    integer, parameter :: blockDim = 256
    integer :: natoms, nframes
    real(4), allocatable :: x(:,:)
    real(4), allocatable :: y(:,:)
    real(4), allocatable :: z(:,:)
    double precision :: xbox, ybox, zbox, cut
    double precision :: vol, del, s2, s2bond
    double precision, allocatable :: g(:)
    double precision :: rho, pi, const, nideal, rf
    real(4) :: start, tread, tpairs, finish

    open(23, file='RDF.dat', status='unknown')
    call readdcd(maxframes, maxatoms, x, y, z, xbox, ybox, zbox, natoms, nframes)
    call pair_calculation<<<1, blockDim>>>(x, y, z, nframes, natoms, nbin, g, xbox, del)
    call entropy_calculation<<<1, blockDim>>>(g, nbin, rho, del, s2, s2bond)
    ! ... (unchanged)
end program rdf
